In [27]:
import torch
import matplotlib.pyplot as plt
import pickle
import gzip

## Dataset download

In [8]:
!wget http://deeplearning.net/data/mnist/mnist.pkl.gz
!mkdir datasets
!mv mnist.pkl.gz datasets/

--2020-10-03 10:14:22--  http://deeplearning.net/data/mnist/mnist.pkl.gz
Resolving deeplearning.net (deeplearning.net)... 132.204.26.28
Connecting to deeplearning.net (deeplearning.net)|132.204.26.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16168813 (15M) [application/x-gzip]
Saving to: ‘mnist.pkl.gz’

mnist.pkl.gz        100%[===================>]  15,42M  3,84MB/s    in 5,5s    

2020-10-03 10:14:28 (2,81 MB/s) - ‘mnist.pkl.gz’ saved [16168813/16168813]

zsh:1: command not found: tgzip


In [17]:
PATH = 'datasets/mnist.pkl.gz'
with gzip.open(PATH, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [21]:
x_train, y_train, x_valid, y_valid = map(tensor, (x_train, y_train, x_valid, y_valid))

In [22]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [24]:
x_train.min(), x_train.max(), x_valid.min(), x_valid.max()

(tensor(0.), tensor(0.9961), tensor(0.), tensor(0.9961))

In [25]:
y_train.min(), y_train.max(), y_valid.min(), y_valid.max()

(tensor(0), tensor(9), tensor(0), tensor(9))

## Initial model

In [28]:
weights = torch.randn(784, 10)
bias = torch.randn(10)

### Matrix multiplication

In [32]:
def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for k in range(ac):
        for i in range(ar):
            for j in range(bc):
                c[i, j] += a[i, k] * b[k, j]
    return c

In [30]:
m1 = x_valid[:5]
m2 = weights
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [37]:
%time t1 = matmul(m1, m2); t1.shape

CPU times: user 720 ms, sys: 4.96 ms, total: 725 ms
Wall time: 726 ms


torch.Size([5, 10])

725ms with only 5 rows. With the entire 50000 rows equals *(145 ms * 50000 rows)* that is **approximately 2 hours**.


The way to make Python faster, is remove Python.

In [38]:
len(x_train)

50000

### Pytorch elementwise operations

Operations (+, -, *, /, >, <, ==)

In [65]:
a = torch.randn(10)
b = torch.randn(10)

In [66]:
(a < b)

tensor([ True, False, False,  True,  True, False,  True,  True, False,  True])

In [67]:
(a < b).float().mean()

tensor(0.6000)

60% of **a** are less than **b**

**Frobenius Norm (Matrix Normalization)**

The Frobenius norm, sometimes also called the Euclidean norm (a term unfortunately also used for the vector L^2-norm), is matrix norm of an m×n matrix A defined as the square root of the sum of the absolute squares of its elements. [Wolfram](https://mathworld.wolfram.com/FrobeniusNorm.html)

$$\|A\|_\text{F} = \sqrt{\sum_{i=1}^m \sum_{j=1}^n |a_{ij}|^2} $$

The Frobenius norm can also be considered as a vector norm. 

In [69]:
m = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float32)

In [95]:
def frobeniusNorm(x):
    a = 0.
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            a += x[i, j] * x[i, j] #sum
    return a ** (1/2) #sqrt
%time frobeniusNorm(m)

CPU times: user 468 µs, sys: 209 µs, total: 677 µs
Wall time: 522 µs


tensor(16.8819)

or 

In [94]:
%time (m*m).sum().sqrt()

CPU times: user 256 µs, sys: 130 µs, total: 386 µs
Wall time: 259 µs


tensor(16.8819)